# Part 06

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc
from collections import Counter, defaultdict

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 70.9 MB/s eta 0:00:00


In [4]:
pip install factor_analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor_analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42655 sha256=056ec5eb2ecd875039748be35c308175b2a15f6da71c68600101d61609edc50a
  Stored in directory: /root/.cache/pip/wheels/a2/af/06/f4d4ed4d9d714fda437fb1583629417319603c2266e7b233cc
Successfully built factor_analyzer


In [5]:
import sys
sys.path.append("/content/drive/MyDrive/week_hands_on_project5")

from whiskey.model.recommender_model2 import WhiskyRecommender

In [6]:
import pickle

with open("/content/drive/MyDrive/week_hands_on_project5/whiskey/model/whisky_recommender2.pkl", "rb") as f:
    recommender = pickle.load(f)

df = recommender.df

In [7]:
print(type(recommender))
print("df shape:", recommender.df.shape)

print("Taste ready:", recommender.fa_model is not None, recommender.knn_taste is not None)
print("Meta ready:", recommender.knn_meta is not None, recommender.X_meta is not None)

print("df_taste size:", 0 if recommender.df_taste is None else len(recommender.df_taste))
print("meta dim:", None if recommender.X_meta is None else recommender.X_meta.shape)

<class 'whiskey.model.recommender_model2.WhiskyRecommender'>
df shape: (1083, 61)
Taste ready: True True
Meta ready: True True
df_taste size: 618
meta dim: (1083, 102)


# 평가

## ⒈ 거리 기반 성능 검증

In [8]:

def neighbor_sanity_check(
    rec,
    n_samples=200,
    k=20,
    random_state=42,
    smoke_threshold=3.0,
    add_random_baseline=True,
    metrics=("taste_gap_mean", "taste_gap_p95", "meta_dist_mean", "improvement"),
    return_summary=True,
    return_worst=True,
    return_group=True,
    # ✅ worst/해석에 같이 붙일 메타 플래그
    extra_cols=("is_independent", "is_vintage", "has_age", "has_bottling_decade"),
    # ✅ worst 뽑는 기준을 명시적으로 선택 가능
    worst_by="taste_gap_p95",  # {"taste_gap_p95","taste_gap_mean","improvement"}
    worst_top_n=10,
):
    """
    거리 기반 성능 sanity check (Taste / Meta / Random Baseline)

    metrics 옵션:
    - "taste_gap_mean" : anchor 기준 style gap 평균
    - "taste_gap_std"  : (anchor 내부) 이웃 gap의 표준편차
    - "taste_gap_p95"  : (anchor 내부) gap 95% 분위
    - "taste_gap_max"  : (anchor 내부) gap 최대
    - "taste_dist_mean"/"taste_dist_std"/"taste_dist_p95"
    - "meta_dist_mean"/"meta_dist_std"/"meta_dist_p95"
    - "improvement" : 랜덤 대비 개선율(랜덤 baseline 필요)

    반환:
    - rep (row=anchor 단위)
    - (선택) summary: rep.describe()
    - (선택) worst: worst_by 기준 상위/하위
    - (선택) group_summary: smoky_flag 그룹 요약
    """

    rng = np.random.default_rng(random_state)

    df = rec.df.copy()
    style_cols = rec.style_cols

    # -----------------------------
    # pool 구성 (taste space 가능한 제품만)
    # -----------------------------
    taste_pool = rec.df_taste.dropna(subset=["FA1", "FA2"])
    if taste_pool.empty:
        raise ValueError("taste_pool이 비었습니다. FA1/FA2 생성 여부 확인 필요")

    sample_idx = rng.choice(
        taste_pool.index.to_numpy(),
        size=min(n_samples, len(taste_pool)),
        replace=False,
    )
    baseline_pool_idx = taste_pool.index.to_numpy()

    # -----------------------------
    # 어떤 계산이 필요한지 체크
    # -----------------------------
    need_taste_gap = any(m.startswith("taste_gap_") for m in metrics) or ("improvement" in metrics)
    need_taste_dist = any(m.startswith("taste_dist_") for m in metrics)
    need_meta = any(m.startswith("meta_dist_") for m in metrics)
    need_baseline = ("improvement" in metrics)

    if need_baseline and (not add_random_baseline):
        raise ValueError("metrics에 'improvement'가 포함되었는데 add_random_baseline=False 입니다.")

    # -----------------------------
    # main loop
    # -----------------------------
    rows = []
    for idx in sample_idx:
        anchor = df.loc[idx]

        # (1) taste neighbors (필요할 때만)
        tnb = None
        if need_taste_gap or need_taste_dist:
            point = anchor[["FA1", "FA2"]].to_numpy(dtype=float)
            tnb = rec._knn_taste_point(point, top_k=k + 1)
            # self 포함 제거 후 k개
            tnb = tnb[tnb.index != idx].head(k).copy()

        # (2) row 기본 + 해석용 메타 플래그 포함
        row = {
            "idx": idx,
            "name": anchor.get("name", None),
            "style_smoke": anchor.get("style_smoke", np.nan),
        }
        for c in extra_cols:
            row[c] = anchor.get(c, np.nan)

        row["smoky_flag"] = bool(row["style_smoke"] >= smoke_threshold) if pd.notna(row["style_smoke"]) else False

        # -----------------------------
        # A) taste gap stats (원본 4축 거리)
        # -----------------------------
        if need_taste_gap and (tnb is not None) and (not tnb.empty) and all(c in df.columns for c in style_cols):
            a_style = anchor[style_cols].to_numpy(dtype=float)
            nb_style = tnb[style_cols].to_numpy(dtype=float)
            style_l2 = np.linalg.norm(nb_style - a_style.reshape(1, -1), axis=1)

            # ✅ improvement 단독이어도 mean 필요
            if ("taste_gap_mean" in metrics) or ("improvement" in metrics):
                row["taste_style_gap_mean"] = float(np.nanmean(style_l2))
            if "taste_gap_std" in metrics:
                row["taste_style_gap_std"] = float(np.nanstd(style_l2))
            if "taste_gap_p95" in metrics:
                row["taste_style_gap_p95"] = float(np.nanpercentile(style_l2, 95))
            if "taste_gap_max" in metrics:
                row["taste_style_gap_max"] = float(np.nanmax(style_l2))
        else:
            if ("taste_gap_mean" in metrics) or ("improvement" in metrics):
                row["taste_style_gap_mean"] = np.nan
            if "taste_gap_std" in metrics:
                row["taste_style_gap_std"] = np.nan
            if "taste_gap_p95" in metrics:
                row["taste_style_gap_p95"] = np.nan
            if "taste_gap_max" in metrics:
                row["taste_style_gap_max"] = np.nan

        # -----------------------------
        # B) taste distance stats (FA 공간 거리)
        # -----------------------------
        if need_taste_dist and (tnb is not None) and (not tnb.empty):
            if "taste_distance" in tnb.columns:
                taste_dist = pd.to_numeric(tnb["taste_distance"], errors="coerce")
            else:
                taste_dist = pd.Series(np.nan, index=tnb.index)

            if "taste_dist_mean" in metrics:
                row["taste_dist_mean"] = float(np.nanmean(taste_dist))
            if "taste_dist_std" in metrics:
                row["taste_dist_std"] = float(np.nanstd(taste_dist))
            if "taste_dist_p95" in metrics:
                row["taste_dist_p95"] = float(np.nanpercentile(taste_dist.dropna(), 95)) if taste_dist.notna().any() else np.nan
        else:
            if "taste_dist_mean" in metrics:
                row["taste_dist_mean"] = np.nan
            if "taste_dist_std" in metrics:
                row["taste_dist_std"] = np.nan
            if "taste_dist_p95" in metrics:
                row["taste_dist_p95"] = np.nan



        # -----------------------------
        # D) random baseline improvement
        # -----------------------------
        if add_random_baseline and need_baseline and all(c in df.columns for c in style_cols):
            knn_gap = row.get("taste_style_gap_mean", np.nan)
            cand = baseline_pool_idx[baseline_pool_idx != idx]

            if len(cand) >= k:
                rand_idx = rng.choice(cand, size=k, replace=False)
                rand_nb = df.loc[rand_idx, style_cols].to_numpy(dtype=float)

                a_style = anchor[style_cols].to_numpy(dtype=float)
                rand_l2 = np.linalg.norm(rand_nb - a_style.reshape(1, -1), axis=1)
                rand_gap_mean = float(np.nanmean(rand_l2))

                row["random_style_gap_mean"] = rand_gap_mean

                # 1 - (knn/rand): +면 개선
                if rand_gap_mean > 0 and not np.isnan(knn_gap):
                    row["knn_vs_random_improvement"] = float(1.0 - (knn_gap / rand_gap_mean))
                else:
                    row["knn_vs_random_improvement"] = np.nan
            else:
                row["random_style_gap_mean"] = np.nan
                row["knn_vs_random_improvement"] = np.nan
        else:
            if "improvement" in metrics:
                row["random_style_gap_mean"] = np.nan
                row["knn_vs_random_improvement"] = np.nan

        rows.append(row)

    rep = pd.DataFrame(rows)

    # -----------------------------
    # outputs
    # -----------------------------
    out = [rep]

    if return_summary:
        summary = rep.describe(include="all").T
        out.append(summary)

    if return_worst:
        # worst 기준 선택
        if worst_by == "taste_gap_p95":
            key = "taste_style_gap_p95"
            ascending = False
        elif worst_by == "taste_gap_mean":
            key = "taste_style_gap_mean"
            ascending = False
        elif worst_by == "improvement":
            key = "knn_vs_random_improvement"
            ascending = True  # improvement는 낮을수록 worst
        else:
            raise ValueError("worst_by는 {'taste_gap_p95','taste_gap_mean','improvement'} 중 하나여야 합니다.")

        if key in rep.columns:
            worst = rep.sort_values(key, ascending=ascending).head(worst_top_n).copy()

            # 보기 좋은 컬럼 순서
            front = [
                "idx", "name",
                "style_smoke", "smoky_flag",
                *list(extra_cols),
                key,
                "random_style_gap_mean",
                "taste_style_gap_mean",
                "taste_style_gap_p95",
                "meta_dist_mean",
            ]
            cols = [c for c in front if c in worst.columns] + [c for c in worst.columns if c not in front]
            worst = worst.loc[:, cols].reset_index(drop=True)
        else:
            worst = rep.head(0)

        out.append(worst)

    if return_group:
        numeric_cols = [
            c for c in rep.columns
            if c not in ["idx", "name"]
            and pd.api.types.is_numeric_dtype(rep[c])
        ]
        group_summary = rep.groupby("smoky_flag")[numeric_cols].describe() if numeric_cols else None
        out.append(group_summary)

    return tuple(out)


rep, summary, worst, group_summary = neighbor_sanity_check(
    rec=recommender,
    n_samples=200,
    k=20,
    random_state=42,
    smoke_threshold=3.0,
    add_random_baseline=True,
    metrics=("taste_gap_mean", "taste_gap_p95", "improvement"),
    return_summary=True,
    return_worst=True,
    return_group=True,
    extra_cols=("is_independent", "is_vintage", "has_age", "has_bottling_decade"),
    worst_by="taste_gap_p95",
    worst_top_n=10,
)

metric_cols = []
for c in [
    "taste_style_gap_mean",
    "taste_style_gap_p95",

    "random_style_gap_mean",
    "knn_vs_random_improvement",
]:
    if c in rep.columns:
        metric_cols.append(c)

kpi = pd.DataFrame({
    "mean": rep[metric_cols].mean(numeric_only=True),
    "std": rep[metric_cols].std(numeric_only=True),
    "median": rep[metric_cols].median(numeric_only=True),
    "p95": rep[metric_cols].quantile(0.95, numeric_only=True),
    "max": rep[metric_cols].max(numeric_only=True),
}).round(4)

display(kpi)

,mean,std,median,p95,max
taste_style_gap_mean,0.4883,0.6428,0.1750,1.6866,3.0686
taste_style_gap_p95,0.8921,0.9736,1.0000,2.4697,4.1231
random_style_gap_mean,2.1405,0.5493,2.0382,3.1430,4.6879
knn_vs_random_improvement,0.8004,0.2456,0.9264,1.0000,1.0000


In [9]:
if "taste_style_gap_mean" in rep.columns and "random_style_gap_mean" in rep.columns and "knn_vs_random_improvement" in rep.columns:
    msg = (
        f"Taste KNN의 anchor 대비 평균 style gap은 {rep['taste_style_gap_mean'].mean():.3f}이며, "
        f"랜덤 이웃의 평균 gap {rep['random_style_gap_mean'].mean():.3f} 대비 "
        f"개선율은 평균 {rep['knn_vs_random_improvement'].mean():.3f} (≈% {(rep['knn_vs_random_improvement'].mean()*100):.1f}) 입니다."
    )
    print(msg)

Taste KNN의 anchor 대비 평균 style gap은 0.488이며, 랜덤 이웃의 평균 gap 2.140 대비 개선율은 평균 0.800 (≈% 80.0) 입니다.


    1. taste_style_gap_mean = 0.49 | random_style_gap_mean =2.14 | knn_vs_random_improvement = 0.8
    
    랜덤 이웃 스타일 차이에 비해 taste knn 이웃 차이를 보면 80% 정도 개선되었다.



In [10]:
#
display(worst)

,idx,name,style_smoke,smoky_flag,is_independent,is_vintage,has_age,has_bottling_decade,taste_style_gap_p95,random_style_gap_mean,taste_style_gap_mean,taste_style_gap_p95,knn_vs_random_improvement
0,292,Inchdairnie Ryelaw 2017 Single GrainBot.2022,0.0,False,0,1,0,1,4.123106,3.348806,2.748448,4.123106,0.179275
1,995,Teeling Blackpitts Peated Single Malt,3.0,True,0,0,0,0,3.741657,2.995835,2.194564,3.741657,0.267462
2,849,Berry Bros & Rudd Islay Blended Malt,3.0,True,1,0,0,0,3.477979,3.569967,2.666948,3.477979,0.252949
3,189,Royal Salute Platinum JubileeQueen Mary Brooch...,3.0,True,0,0,0,0,3.477979,4.057539,3.068562,3.477979,0.243738
4,9,Johnnie Walker Black Label 12 Year Old,3.0,True,0,0,1,0,3.177369,3.297602,1.683216,3.177369,0.489564
5,567,Spey Fumare,3.0,True,0,0,0,0,3.177369,3.549951,1.683216,3.177369,0.525848
6,921,The Six Isles Voyager,3.0,True,0,0,0,0,3.162278,3.671722,1.990732,3.162278,0.457821
7,415,Speyside (GL) 16 Year Old100 Proof Exceptional...,0.0,False,1,0,1,0,3.037083,2.213492,2.010361,3.037083,0.091769
8,1022,West Cork Bog Oak Charred Cask,0.0,False,0,0,0,0,3.037083,2.598324,2.492348,3.037083,0.040786
9,913,Big Peat 12 Year Old,4.0,True,1,0,1,0,2.852837,4.687902,1.947372,2.852837,0.584596


In [11]:
# meta 이웃은 카테고리를 실제로 공유하나?
# 이웃들의 카테고리 일치율로 검증 precision@k
def meta_neighbor_validity(rec, n_samples=300, k=30, random_state=42):
    rng = np.random.default_rng(random_state)
    df = rec.df

    cat_cols = ["whisky_type", "region", "bottler_group", "age_bin", "cask_group"]
    cat_cols = [c for c in cat_cols if c in df.columns]

    idxs = rec._meta_index
    sample = rng.choice(idxs, size=min(n_samples, len(idxs)), replace=False)

    rows = []
    for idx in sample:
        pos = np.where(rec._meta_index == idx)[0]
        if len(pos) == 0:
            continue
        anchor = df.loc[idx]
        center = rec.X_meta[pos[0]]

        nb = rec._knn_meta_point(center, top_k=k+1).copy()
        nb = nb[nb.index != idx].head(k)

        # 카테고리별 일치율
        match_rates = {}
        for c in cat_cols:
            a = anchor.get(c, "Unknown")
            match_rates[f"match_{c}"] = float((nb[c] == a).mean()) if c in nb.columns else np.nan

        rows.append({
            "idx": idx,
            "meta_dist_mean": float(nb["meta_distance"].mean()) if "meta_distance" in nb.columns else np.nan,
            **match_rates
        })

    rep = pd.DataFrame(rows)
    summary = rep.describe().T
    return rep, summary


rep_meta, sum_meta = meta_neighbor_validity(recommender, n_samples=300, k=30)
sum_meta

,count,mean,std,min,25%,50%,75%,max
idx,300.0,522.043333,299.975118,6.000000,278.250000,512.500000,743.250000,1078.000000
meta_dist_mean,300.0,1.882836,1.202587,0.000000,1.034733,1.803651,2.805896,5.007982
match_whisky_type,300.0,0.970444,0.122984,0.333333,1.000000,1.000000,1.000000,1.000000
match_region,300.0,0.932667,0.202051,0.000000,1.000000,1.000000,1.000000,1.000000
match_bottler_group,300.0,0.809889,0.342519,0.000000,0.825000,1.000000,1.000000,1.000000
match_age_bin,300.0,0.811778,0.272447,0.000000,0.750000,0.966667,1.000000,1.000000
match_cask_group,300.0,0.732667,0.329881,0.000000,0.500000,0.900000,1.000000,1.000000


In [12]:
def meta_baseline_compare(rec, n_samples=300, k=30, random_state=42):
    rng = np.random.default_rng(random_state)
    df = rec.df

    cat_cols = ["whisky_type", "region", "bottler_group", "age_bin", "cask_group"]
    cat_cols = [c for c in cat_cols if c in df.columns]

    idxs = rec._meta_index
    sample = rng.choice(idxs, size=min(n_samples, len(idxs)), replace=False)

    rows = []
    for idx in sample:
        anchor = df.loc[idx]

        pos = np.where(rec._meta_index == idx)[0]
        if len(pos) == 0:
            continue

        center = rec.X_meta[pos[0]]
        nb_m = rec._knn_meta_point(center, top_k=k+1).copy()
        nb_m = nb_m[nb_m.index != idx].head(k)

        all_idx = df.index.to_numpy()
        rand_idx = rng.choice(all_idx[all_idx != idx], size=min(k, len(all_idx)-1), replace=False)
        rand = df.loc[rand_idx]

        out = {"idx": idx, "meta_dist_mean": float(nb_m["meta_distance"].mean()) if "meta_distance" in nb_m.columns else np.nan}

        for c in cat_cols:
            a = anchor.get(c, "Unknown")
            knn_rate = float((nb_m[c] == a).mean()) if (c in nb_m.columns and not nb_m.empty) else np.nan
            rnd_rate = float((rand[c] == a).mean()) if c in rand.columns else np.nan
            out[f"match_{c}"] = knn_rate
            out[f"gain_{c}"] = knn_rate - rnd_rate

        rows.append(out)

    rep = pd.DataFrame(rows)
    return rep, rep.describe().T

# 함수 실행
rep_meta, summary_meta = meta_baseline_compare(
    recommender,
    n_samples=300,
    k=30,
    random_state=42
)

# 1) anchor 단위 결과 (앞부분만)
display(rep_meta.head(5))

# 2) 전체 describe 요약
display(summary_meta)


,idx,meta_dist_mean,match_whisky_type,gain_whisky_type,match_region,gain_region,match_bottler_group,gain_bottler_group,match_age_bin,gain_age_bin,match_cask_group,gain_cask_group
0,410,1.112515,1.0,0.533333,1.0,0.566667,1.000000,0.133333,1.000000,0.600000,0.900000,0.300000
1,698,3.038476,1.0,0.633333,1.0,0.700000,0.233333,0.233333,0.766667,0.266667,0.766667,0.166667
2,128,0.000000,1.0,0.733333,1.0,0.733333,1.000000,0.266667,1.000000,0.500000,1.000000,0.266667
3,471,1.583919,1.0,0.566667,1.0,0.700000,1.000000,0.233333,0.600000,0.533333,0.733333,-0.033333
4,85,0.000000,1.0,0.600000,1.0,0.466667,1.000000,0.333333,1.000000,0.600000,1.000000,0.400000


,count,mean,std,min,25%,50%,75%,max
idx,300.0,522.043333,299.975118,6.000000,278.250000,512.500000,743.250000,1078.000000
meta_dist_mean,300.0,1.882836,1.202587,0.000000,1.034733,1.803651,2.805896,5.007982
match_whisky_type,300.0,0.970444,0.122984,0.333333,1.000000,1.000000,1.000000,1.000000
gain_whisky_type,300.0,0.658889,0.150629,0.300000,0.566667,0.633333,0.733333,1.000000
match_region,300.0,0.932667,0.202051,0.000000,1.000000,1.000000,1.000000,1.000000
gain_region,300.0,0.654333,0.197644,-0.200000,0.566667,0.666667,0.766667,1.000000
match_bottler_group,300.0,0.809889,0.342519,0.000000,0.825000,1.000000,1.000000,1.000000
gain_bottler_group,300.0,0.257444,0.169880,-0.266667,0.166667,0.233333,0.300000,0.866667
match_age_bin,300.0,0.811778,0.272447,0.000000,0.750000,0.966667,1.000000,1.000000
gain_age_bin,300.0,0.476556,0.256354,-0.366667,0.333333,0.500000,0.666667,0.966667


| Category        | match@k (mean) | gain@k (mean) | 해석 |
|-----------------|----------------|---------------|------|
| Whisky Type     | **0.97**       | **+0.66**     | 거의 항상 동일 타입 → meta 공간의 핵심 축 |
| Region          | **0.93**       | **+0.65**     | 지역 정보가 안정적으로 유지됨 |
| Age Bin         | 0.81           | +0.48         | 숙성 구간은 중간 수준의 신호 |
| Bottler Group   | 0.81           | +0.26         | 보조 신호로 작동 |
| Cask Group      | 0.73           | +0.22         | 스타일 다양성 확보를 위한 보조 축 |

## ⒉ 제품 노출 편향


- 제품 다양성과 제품 반복성

In [16]:
# - simulate_users_for_exposure: user survey simulation -> exp_df (name, exposure)
# - exposure_metrics: catalog coverage / concentration / gini / uniqueness
# - attach_exposure_to_catalog: join exposure back to catalog df
# - group_exposure_bias_table: group-level bias (exposure share vs catalog share)
# - find_underexposed_groups: under-exposed group shortlist
# - run_exposure_bias_pipeline: one-shot runner (prints + returns dict)

from collections import Counter

# -----------------------------
# 1) Simulation: exposure count
# -----------------------------
def simulate_users_for_exposure(
    rec,
    n_users=500,
    top_k=30,
    families_pool=None,
    random_state=42,
):
    """
    설문 입력(4축 + family 토글)을 랜덤 샘플링해서 추천을 n_users번 실행.
    그 결과로 노출(추천 등장) 분포(exp_df)를 만든다.

    반환:
    - exp_df: columns=["name","exposure"] (노출 수 내림차순)
    - per_user_lists: 유저별 top_k 추천 name 리스트들
    """
    rng = np.random.default_rng(random_state)

    def sample_style(col):
        s = rec.df[col].dropna()
        return float(rng.choice(s.values)) if not s.empty else None

    if families_pool is None:
        families_pool = [c.replace("_family", "") for c in getattr(rec, "family_cols", [])]

    exposures = Counter()
    per_user_lists = []

    for _ in range(n_users):
        b = sample_style("style_body")
        r = sample_style("style_richness")
        s = sample_style("style_smoke")
        sw = sample_style("style_sweetness")

        n_f = int(rng.integers(0, 3))
        fams = (
            list(rng.choice(families_pool, size=min(n_f, len(families_pool)), replace=False))
            if (n_f > 0 and len(families_pool) > 0)
            else []
        )

        out = rec.recommend_from_survey(
            style_body=b,
            style_richness=r,
            style_smoke=s,
            style_sweetness=sw,
            selected_families=fams,
            final_k=top_k,
            random_state=int(rng.integers(0, 1_000_000_000)),
        )

        final_df = out.get("final_candidates", pd.DataFrame())
        if final_df is None or final_df.empty or "name" not in final_df.columns:
            continue

        names = final_df["name"].head(top_k).tolist()
        if not names:
            continue

        per_user_lists.append(names)
        exposures.update(names)

    exp_df = pd.DataFrame(exposures.items(), columns=["name", "exposure"])
    exp_df = exp_df.sort_values("exposure", ascending=False).reset_index(drop=True)

    return exp_df, per_user_lists


def exposure_metrics(exp_df, per_user_lists, catalog_size=None, topk=30):
    """
    노출 편향 지표:
    - catalog_coverage: 노출된 고유 상품 수 / 카탈로그 크기
    - exposure_share_top_{1,5,10}pct: 상위 p% 상품이 차지하는 노출 비중
    - gini_exposure: 노출 불평등(0=균등, 1=집중)
    - avg_unique_at_k / min_unique_at_k: 유저별 topk 추천 다양성(중복 제거)
    """
    metrics = {}

    if exp_df is None or exp_df.empty:
        return pd.Series(
            {
                "avg_unique_at_k": np.nan,
                "catalog_coverage": 0.0 if catalog_size else np.nan,
                "exposure_share_top_10pct": np.nan,
                "exposure_share_top_1pct": np.nan,
                "exposure_share_top_5pct": np.nan,
                "gini_exposure": np.nan,
                "min_unique_at_k": np.nan,
            }
        )

    total_exposures = float(exp_df["exposure"].sum())
    n_recommended_items = int(len(exp_df))

    metrics["catalog_coverage"] = (n_recommended_items / catalog_size) if catalog_size else np.nan

    def share_top(p):
        n = max(int(len(exp_df) * p), 1)
        return (exp_df["exposure"].head(n).sum() / total_exposures) if total_exposures > 0 else np.nan

    metrics["exposure_share_top_1pct"] = share_top(0.01)
    metrics["exposure_share_top_5pct"] = share_top(0.05)
    metrics["exposure_share_top_10pct"] = share_top(0.10)

    # Gini
    x = exp_df["exposure"].to_numpy(dtype=float)
    if x.sum() == 0 or len(x) == 0:
        gini = np.nan
    else:
        x = np.sort(x)
        n = len(x)
        cumx = np.cumsum(x)
        gini = (n + 1 - 2 * (cumx / cumx[-1]).sum()) / n
    metrics["gini_exposure"] = gini

    uniq_counts = [len(set(lst[:topk])) for lst in per_user_lists if lst]
    metrics["avg_unique_at_k"] = float(np.mean(uniq_counts)) if uniq_counts else np.nan
    metrics["min_unique_at_k"] = float(np.min(uniq_counts)) if uniq_counts else np.nan

    return pd.Series(metrics).sort_index()


# ---------------------------------------------------
# 2) Join exposure -> catalog and analyze group bias
# ---------------------------------------------------
def attach_exposure_to_catalog(rec, exp_df):
    """
    exp_df(name, exposure)를 rec.df(name 포함)에 붙여서,
    제품 단위 exposure=0 포함 전체 카탈로그 df를 만든다.
    """
    df = rec.df.copy()
    out = df.merge(exp_df, on="name", how="left")
    out["exposure"] = out["exposure"].fillna(0).astype(int)
    return out


def group_exposure_bias_table(
    df_exp,       # attach_exposure_to_catalog 결과
    group_col,
    min_items=10,
    top_n=15,
):
    """
    그룹별 노출 편향 테이블 생성 (노출 점유율 vs 카탈로그 점유율)

    반환:
    - over_df: bias_pp(+) 큰 그룹 top_n
    - under_df: bias_pp(-) 큰 그룹 top_n
    - full_df: 전체 그룹 결과(정렬 전)
    """
    if group_col not in df_exp.columns:
        raise ValueError(f"{group_col} not found")

    tmp = df_exp.copy()
    tmp[group_col] = tmp[group_col].fillna("Unknown")

    total_items = len(tmp)
    total_exposure = tmp["exposure"].sum()

    g = tmp.groupby(group_col, dropna=False).agg(
        n_items=("name", "count"),
        exposure_sum=("exposure", "sum"),
        exposure_per_item=("exposure", "mean"),
        median_exposure=("exposure", "median"),
        p90_exposure=("exposure", lambda x: float(np.quantile(x, 0.9))),
        zero_exposure_rate=("exposure", lambda x: float((x == 0).mean())),
    ).reset_index()

    g["catalog_share"] = g["n_items"] / total_items if total_items > 0 else np.nan
    g["exposure_share"] = g["exposure_sum"] / total_exposure if total_exposure > 0 else np.nan
    g["bias_pp"] = (g["exposure_share"] - g["catalog_share"]) * 100  # percentage points

    # 작은 그룹은 노이즈가 크니 제외
    g = g[g["n_items"] >= min_items].copy()

    over_df = g.sort_values("bias_pp", ascending=False).head(top_n).copy()
    under_df = g.sort_values("bias_pp", ascending=True).head(top_n).copy()

    # 보기 좋게 % 변환/라운딩
    def pretty(df_):
        df_ = df_.copy()
        df_["catalog_share(%)"] = (df_["catalog_share"] * 100).round(2)
        df_["exposure_share(%)"] = (df_["exposure_share"] * 100).round(2)
        df_["bias_pp"] = df_["bias_pp"].round(2)
        df_["exposure_per_item"] = df_["exposure_per_item"].round(2)
        df_["zero_exposure_rate(%)"] = (df_["zero_exposure_rate"] * 100).round(1)
        keep = [
            group_col,
            "n_items",
            "catalog_share(%)",
            "exposure_sum",
            "exposure_share(%)",
            "bias_pp",
            "exposure_per_item",
            "zero_exposure_rate(%)",
        ]
        return df_[keep]

    return pretty(over_df), pretty(under_df), g


def find_underexposed_groups(full_g, group_col, top_n=10, min_items=10):
    """
    소외 그룹(under-exposed) 후보를 score로 정렬.
    score가 높을수록 소외 가능성이 큼.

    사용되는 시그널:
    - bias_pp가 음수(카탈로그 비중 대비 노출이 부족)
    - zero_exposure_rate가 높음(그룹 내 0노출 제품이 많음)
    - exposure_per_item이 낮음
    """
    g = full_g.copy()
    g = g[g["n_items"] >= min_items].copy()

    # score: bias(음수) + zero_rate + low exposure
    # zero_exposure_rate는 0~1 이므로 가중
    g["under_score"] = (-g["bias_pp"]) + (g["zero_exposure_rate"] * 50) + (1 / (g["exposure_per_item"] + 1e-6))

    out = g.sort_values("under_score", ascending=False).head(top_n).copy()

    out["catalog_share(%)"] = (out["catalog_share"] * 100).round(2)
    out["exposure_share(%)"] = (out["exposure_share"] * 100).round(2)
    out["bias_pp"] = out["bias_pp"].round(2)
    out["exposure_per_item"] = out["exposure_per_item"].round(2)
    out["zero_exposure_rate(%)"] = (out["zero_exposure_rate"] * 100).round(1)
    out["under_score"] = out["under_score"].round(2)

    keep = [
        group_col,
        "n_items",
        "catalog_share(%)",
        "exposure_share(%)",
        "bias_pp",
        "exposure_per_item",
        "zero_exposure_rate(%)",
        "under_score",
    ]
    return out[keep]


# -----------------------------
# 3) One-shot pipeline runner
# -----------------------------
def run_exposure_bias_pipeline(
    rec,
    n_users=500,
    top_k=30,
    random_state=42,
    group_cols=None,
    min_items=10,
    top_n=15,
    under_top_n=10,
):
    """
    한 번에:
    1) 시뮬레이션으로 exp_df 생성
    2) metrics 계산
    3) 그룹별 편향/소외 테이블 출력
    4) 결과 dict 반환
    """
    if group_cols is None:
        group_cols = [
            "whisky_type",
            "country",
            "region",
            "bottler_group",
            "age_bin",
            "cask_group",
        ]

    # 1) simulate exposures
    exp_df, per_user_lists = simulate_users_for_exposure(
        rec,
        n_users=n_users,
        top_k=top_k,
        families_pool=None,
        random_state=random_state,
    )

    # 2) product-level metrics
    catalog_size = len(rec.df)
    metrics = exposure_metrics(exp_df, per_user_lists, catalog_size=catalog_size, topk=top_k)

    # 3) join to catalog
    df_exp = attach_exposure_to_catalog(rec, exp_df)

    # 4) group analyses (available only)
    available = [c for c in group_cols if c in df_exp.columns]
    results = {
        "exp_df": exp_df,
        "per_user_lists": per_user_lists,
        "metrics": metrics,
        "df_exp": df_exp,
        "group_results": {},
        "available_group_cols": available,
    }

    print("=== Exposure Metrics (product-level) ===")
    display(metrics.to_frame("value"))

    print("\n=== Top Exposed Products (head) ===")
    display(exp_df.head(15))

    if not available:
        print("\n[WARN] group_cols 중 df에 존재하는 컬럼이 없습니다. df.columns 확인하세요.")
        return results

    for col in available:
        over_df, under_df, full_g = group_exposure_bias_table(
            df_exp, col, min_items=min_items, top_n=top_n
        )
        under_scored = find_underexposed_groups(
            full_g, col, top_n=under_top_n, min_items=min_items
        )

        results["group_results"][col] = {
            "over": over_df,
            "under": under_df,
            "under_scored": under_scored,
            "full": full_g,
        }

        print(f"\n\n===== GROUP BIAS: [{col}] =====")
        print("[Over-exposed groups] (bias_pp 큰 순)")
        display(over_df)

        print("[Under-exposed groups] (bias_pp 작은 순)")
        display(under_df)

        print("[Under-exposed candidates] (score 기반)")
        display(under_scored)

    return results


# ================================================

results = run_exposure_bias_pipeline(
    recommender,
    n_users=500,
    top_k=30,
    random_state=42,
    group_cols=["whisky_type","country","region","bottler_group","age_bin","cask_group"],
    min_items=10,
    top_n=15,
    under_top_n=10
)

metrics = results["metrics"]
exp_df = results["exp_df"]
group_tables = results["group_results"]



=== Exposure Metrics (product-level) ===


,value
avg_unique_at_k,30.000000
catalog_coverage,0.736842
exposure_share_top_10pct,0.394667
exposure_share_top_1pct,0.069133
exposure_share_top_5pct,0.246400
gini_exposure,0.559874
min_unique_at_k,30.000000



=== Top Exposed Products (head) ===


,name,exposure
0,Mortlach 12 Year OldThe Wee Witchie,198
1,Glenfiddich Project XXExperimental Series,184
2,Cardhu Gold ReserveGame of Thrones House Targa...,161
3,Old Ballantruan,153
4,Glenlivet Founder's Reserve,119
5,Glenfiddich 30 Year OldSuspended Time Re-imagi...,114
6,Cragganmore 12 Year Old,108
7,Dalwhinnie 15 Year Old,102
8,Glenfiddich 15 Year Old Solera,102
9,Glenallachie 8 Year Old,101




===== GROUP BIAS: [whisky_type] =====
[Over-exposed groups] (bias_pp 큰 순)


...

,cask_group,n_items,catalog_share(%),exposure_sum,exposure_share(%),bias_pp,exposure_per_item,zero_exposure_rate(%)
8,unknown,752,69.44,11248,74.14,4.70,14.96,23.1
9,wine_port,41,3.79,674,4.44,0.66,16.44,17.1
6,rum,12,1.11,177,1.17,0.06,14.75,16.7
5,others,19,1.75,265,1.75,-0.01,13.95,26.3
7,sherry,142,13.11,1866,12.30,-0.81,13.14,28.9
1,bourbon,98,9.05,771,5.08,-3.97,7.87,34.7


[Under-exposed groups] (bias_pp 작은 순)


,cask_group,n_items,catalog_share(%),exposure_sum,exposure_share(%),bias_pp,exposure_per_item,zero_exposure_rate(%)
1,bourbon,98,9.05,771,5.08,-3.97,7.87,34.7
7,sherry,142,13.11,1866,12.30,-0.81,13.14,28.9
5,others,19,1.75,265,1.75,-0.01,13.95,26.3
6,rum,12,1.11,177,1.17,0.06,14.75,16.7
9,wine_port,41,3.79,674,4.44,0.66,16.44,17.1
8,unknown,752,69.44,11248,74.14,4.70,14.96,23.1


[Under-exposed candidates] (score 기반)


,cask_group,n_items,catalog_share(%),exposure_share(%),bias_pp,exposure_per_item,zero_exposure_rate(%),under_score
1,bourbon,98,9.05,5.08,-3.97,7.87,34.7,21.44
7,sherry,142,13.11,12.30,-0.81,13.14,28.9,15.33
5,others,19,1.75,1.75,-0.01,13.95,26.3,13.24
6,rum,12,1.11,1.17,0.06,14.75,16.7,8.34
9,wine_port,41,3.79,4.44,0.66,16.44,17.1,7.94
8,unknown,752,69.44,74.14,4.70,14.96,23.1,6.94


## ⒊ 모델 안정성

In [17]:
def jaccard(a, b):
    a, b = set(a), set(b)
    return len(a & b) / len(a | b) if len(a | b) > 0 else np.nan


def stability_test_same_input(rec, user_input, n_runs=30, top_k=30):
    """
    같은 입력을 random_state만 바꿔서 여러 번 실행.
    - TopK 결과들의 pairwise Jaccard 평균/분산
    - 가장 자주 등장하는 아이템(편향) 점검
    """
    lists = []
    for i in range(n_runs):
        out = rec.recommend_from_survey(
            **user_input,
            final_k=top_k,
            random_state=1000 + i
        )
        df_final = out.get("final_candidates", pd.DataFrame())
        if df_final is None or df_final.empty:
            continue
        lists.append(df_final["name"].head(top_k).tolist())

    # pairwise jaccard
    sims = []
    for i in range(len(lists)):
        for j in range(i+1, len(lists)):
            sims.append(jaccard(lists[i], lists[j]))

    # 등장 빈도(상위권이 고정되는지)
    freq = Counter([x for lst in lists for x in lst])
    freq_df = pd.DataFrame(freq.items(), columns=["name", "count_in_runs"])
    freq_df = freq_df.sort_values("count_in_runs", ascending=False).reset_index(drop=True)

    return {
        "n_effective_runs": len(lists),
        "jaccard_mean": float(np.mean(sims)) if sims else np.nan,
        "jaccard_std": float(np.std(sims)) if sims else np.nan,
        "most_frequent_items": freq_df.head(20),
        "all_lists": lists
    }


# 예시 입력(너 UI 값으로 바꿔도 됨)
user_input = dict(
    style_body=3.0,
    style_richness=3.0,
    style_smoke=2.0,
    style_sweetness=2.0,
    selected_families=["fruity"]  # 없으면 []
)

stab = stability_test_same_input(recommender, user_input, n_runs=30, top_k=30)
stab["n_effective_runs"], stab["jaccard_mean"], stab["jaccard_std"]

(30, 0.4097380405847556, 0.044148334982149276)